# Supporting Microsoft’s GraphRAG: Part 3

To run this Jupyter Notebook, you can download the original `.ipynb` file from [msft_graphrag_3.ipynb](https://github.com/xuanleilin/tigergraphx/tree/main/docs/graphrag/msft_graphrag_3.ipynb).

---

## Get the Graph from TigerGraph

In [1]:
from tigergraphx import Graph, TigerGraphConnectionConfig
connection = TigerGraphConnectionConfig.ensure_config({
    "host": "http://127.0.0.1",
    "username": "tigergraph",
    "password": "tigergraph",
})
G = Graph.from_db("GraphRAG", connection)

---

## Hybrid Retrieval
TigerGraph offers two flexible ways to perform hybrid retrieval, allowing you to extract relevant graph and vector data efficiently for GraphRAG workflows.
### Using TigerGraphX
TigerGraphX offers an intuitive, Python-native interface for hybrid retrieval, ideal for developers seeking simplicity and ease of use. 

**Key Advantage**: Minimal learning curve with high-level Python APIs, seamlessly integrated with existing workflows.

Below are some illustrative examples.

#### Retrieve Nodes with Specific Attributes
You can use the following code to fetch up to two nodes of type "Entity" and display their "id," "entity_type," and "description" attributes.

In [2]:
G.get_nodes(
    node_type="Entity",
    return_attributes=["id", "name", "entity_type", "description"],
    limit=2,
)

,id,name,entity_type,description
0,7b7427c9d4a943f09fe17738ebb6cfe6,MRSA,EVENT,"Methicillin-resistant Staphylococcus aureus, a..."
1,7150e8aec7644dd99d509dac200f138d,UNIVERSAL PLASMA,ORGANIZATION,A project receiving funding from the Defense H...


#### Retrieve Neighbors with Specific Attributes

The following code demonstrates how to fetch neighbors of specific nodes. In this example, the query retrieves neighbors connected to the given `start_nodes` of type `"Entity"` through the edge type `"community_contains_entity"`. The attributes `"id"`, `"title"`, and `"full_content"` of the neighbors are returned.

In [3]:
start_nodes = ["7373c84a439841d580b4650dac71136f", "4a5ddbde3f354a79bb5ae3436ab67d25"]
G.get_neighbors(
    start_nodes=start_nodes,
    start_node_type="Entity",
    edge_types="community_contains_entity",
    return_attributes=["id", "title", "full_content"],
)

,id,title,full_content
0,10,Community 10,# Innovations in Preventing Contrast-Induced N...
1,1,Community 1,# CytoSorb and Its Global Healthcare Impact\n\...
2,30,Community 30,# CytoSorb and Global Healthcare Impact\n\nThi...


#### Retrieve Top-K Using TigerVector's Vector Search Capability [Planned Feature]

---

### Using GSQL
For developers seeking fine-grained control or complex retrieval logic, GSQL offers unmatched flexibility. As TigerGraph's built-in query language, GSQL empowers you to perform advanced graph data analysis. For more details, see the [official documentation](https://docs.tigergraph.com/gsql-ref/4.1/intro).

**Key Advantage:** Supports complex logic, customization, and direct interaction with TigerGraph’s powerful query engine.

1. Use an LLM to convert the query into an embedding.  
2. Write a GSQL query to retrieve the top-K similar objects and their neighbors, combining structured and vector-based retrieval:

```SQL
CREATE OR REPLACE QUERY query (List<float> embedding, int k) {
  Nodes = TopKVectorSearch({Entity.entity_embedding}, embedding, k);

  Neighbors =
    SELECT t
    FROM Nodes:s -(community_contains_entity:e)- :t;

  PRINT Neighbors[Neighbors.id, Neighbors.title, Neighbors.full_content];
}
```


---
## Context Building: Writing Custom Context Builders

Context builders play a vital role in graph-powered RAG workflows. They transform retrieved graph data into structured, meaningful contexts for tasks such as interactions with LLMs)\.

TigerGraphX simplifies this process by offering the flexible `BaseContextBuilder` class, which allows developers to define custom logic for context building.

### Key Features of `BaseContextBuilder`

The `BaseContextBuilder` class in TigerGraphX provides a strong foundation for creating custom context builders, offering:

- **Core Abstraction**: A reusable framework for building context logic.
- **Customizable Design**: Extensibility for implementing both global and query-specific context generation.

---

### Key Components

1. **Abstract Method - `build_context`**:  
   Subclasses must implement this method to define the logic for constructing context.

   ```python
   @abstractmethod
   async def build_context(self, *args, **kwargs) -> str | List[str]:
       """Abstract method to build context."""
       pass
   ```

2. **Batching and Retrieval Methods**:
   - **`batch_and_convert_to_text`**: Formats graph data into token-aware text.
   - **`retrieve_top_k_objects`**: Efficiently retrieves top-K objects for query-based context.

---

### Example: Global Context Builder

In [4]:
import tiktoken
from typing import Optional, List
from tigergraphx.graphrag import BaseContextBuilder
from tigergraphx.core import Graph
class GlobalContextBuilder(BaseContextBuilder):
    def __init__(
        self,
        graph: Graph,
        token_encoder: Optional[tiktoken.Encoding] = None,
    ):
        """Initialize LocalContextBuilder with graph config and token encoder."""
        super().__init__(
            graph=graph,
            single_batch=False,
            token_encoder=token_encoder,
        )
    async def build_context(self) -> str | List[str]:
        """Build local context."""
        context: List[str] = []
        config = {
            "max_tokens": 12000,
            "section_name": "Communities",
            "return_attributes": ["id", "rank", "title", "full_content"],
            "limit": 1000,
        }
        df = self.graph.get_nodes(
            node_type="Community",
            return_attributes=config["return_attributes"],
            limit=config["limit"],
        )
        if df is not None:
            text_context = self.batch_and_convert_to_text(
                graph_data=df,
                max_tokens=config["max_tokens"],
                single_batch=self.single_batch,
                section_name=config["section_name"],
            )
            context.extend(
                text_context if isinstance(text_context, list) else [text_context]
            )
        return context

Here’s how you can utilize the custom global context builder:

In [5]:
global_context_builder = GlobalContextBuilder(G)
context_list = await global_context_builder.build_context()
# Print the first 1000 characters for easier visualization of long text
print(context_list[0][:1000])

-----Communities-----
id|rank|title|full_content
33|8.5|Community 33|# CytoSorb and Cytokine Storm Management in Brain Dead Organ Donors\n\nThis report focuses on the application of CytoSorb technology by CytoSorbents in managing cytokine storms, particularly in the context of brain dead organ donors. The relationship between cytokine storms and the viability of organs for transplantation highlights a critical area of medical intervention.\n\n## CytoSorb's role in cytokine storm management\n\nCytoSorb is utilized in clinical situations characterized by elevated cytokine levels, including cytokine storms, which are critical conditions that can lead to multiple organ failure. The technology's application to cytokine storm management represents a significant advancement in treating conditions that result in high cytokine levels. This is particularly relevant in the context of brain dead organ donors, where cytokine storms can reduce the viability of organs for transplantation. [Data: Rela

---

### Example: Local Context Builder

To understand the functionality of the `LocalContextBuilder` class, let's review the key code from its `build_context` method.

![](https://github.com/xuanleilin/tigergraphx/blob/main/docs/images/graphrag/local_context_builder.png?raw=true)

```
# Retrieve top-k objects
top_k_objects: List[str] = await self.retrieve_top_k_objects(query, k=k)
...
# Iterate over different neighbor types
for neighbor in neighbor_types:
    df = self.graph.get_neighbors(...)
    if df is not None:
        text_context = self.batch_and_convert_to_text(...)
        context.extend(
            text_context if isinstance(text_context, list) else [text_context]
        )
return "\n\n".join(context)
```

For full implementations of different context builders, refer to the following links:
- [LocalContextBuilder Code](https://github.com/xuanleilin/tigergraphx/blob/main/applications/msft_graphrag/query/context_builder/local_context_builder.py)


Here’s how you can utilize the custom local context builder:

```python
local_builder = LocalContextBuilder(graph=graph, search_engine=search_engine)
local_context = await local_builder.build_context(query="What are the main topics discussed in the article?")
```

---

## Integrate with LLM

After successfully building context from TigerGraph, the final step is integrating it with LLMs, including chat models and embedding models.

We have provided an example implementation, which you can find here: [Example Code](https://github.com/xuanleilin/tigergraphx/blob/main/applications/msft_graphrag/query/graphrag.py).

### Workflow Overview

The integration process follows the workflow illustrated below:

![](https://github.com/xuanleilin/tigergraphx/blob/main/docs/images/graphrag/querying.png?raw=true)

---

## What’s Next?

- [API Reference](../../reference/features_overview): Dive deeper into TigerGraphX APIs.

---

Start transforming your GraphRAG workflows with the power of **TigerGraphX** today!